In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Preprocessing






In [8]:
!pip install h5py

In [9]:
import h5py
import numpy as np


def import_data(every=False):
    if every:
        electrodes = 25
    else:
        electrodes = 22
    X, y = [], []
    for i in range(9):
        A01T = h5py.File('/content/drive/My Drive/data/A0' + str(i + 1) + 'T_slice.mat', 'r')
        X1 = np.copy(A01T['image'])
        X.append(X1[:, :electrodes, :])
        y1 = np.copy(A01T['type'])
        y1 = y1[0, 0:X1.shape[0]:1]
        y.append(np.asarray(y1, dtype=np.int32))

    for subject in range(9):
        delete_list = []
        for trial in range(288):
            if np.isnan(X[subject][trial, :, :]).sum() > 0:
                delete_list.append(trial)
        X[subject] = np.delete(X[subject], delete_list, 0)
        y[subject] = np.delete(y[subject], delete_list)
    y = [y[i] - np.min(y[i]) for i in range(len(y))]
    return X, y


def train_test_subject(X, y, train_all=True, standardize=True):

    l = np.random.permutation(len(X[0]))
    X_test = X[0][l[:50], :, :]
    y_test = y[0][l[:50]]

    if train_all:
        X_train = np.concatenate((X[0][l[50:], :, :], X[1], X[2], X[3], X[4], X[5], X[6], X[7], X[8]))
        y_train = np.concatenate((y[0][l[50:]], y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[8]))

    else:
        X_train = X[0][l[50:], :, :]
        y_train = y[0][l[50:]]

    X_train_mean = X_train.mean(0)
    X_train_var = np.sqrt(X_train.var(0))

    if standardize:
        X_train -= X_train_mean
        X_train /= X_train_var
        X_test -= X_train_mean
        X_test /= X_train_var

    X_train = np.transpose(X_train, (0, 2, 1))
    X_test = np.transpose(X_test, (0, 2, 1))

    return X_train, X_test, y_train, y_test


def train_test_total(X, y, standardize=True):

    X_total = np.concatenate((X[0], X[1], X[2], X[3], X[4], X[5], X[6], X[7], X[8]))
    y_total = np.concatenate((y[0], y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[8]))

    l = np.random.permutation(len(X_total))
    X_test = X_total[l[:50], :, :]
    y_test = y_total[l[:50]]
    X_train = X_total[l[50:], :, :]
    y_train = y_total[l[50:]]

    X_train_mean = X_train.mean(0)
    X_train_var = np.sqrt(X_train.var(0))

    if standardize:
        X_train -= X_train_mean
        X_train /= X_train_var
        X_test -= X_train_mean
        X_test /= X_train_var

    X_train = np.transpose(X_train, (0, 2, 1))
    X_test = np.transpose(X_test, (0, 2, 1))

    return X_train, X_test, y_train, y_test


#Initializers_Testing

In [ ]:
pip install preprocessing

In [ ]:
import tensorflow as tf
import numpy as np
import importlib
import matplotlib.pyplot as plt
import preprocessing
importlib.reload(preprocessing)
from preprocessing import *
import warnings
warnings.filterwarnings('ignore')

In [ ]:
X, y = import_data()
X_train,X_test,y_train,y_test = train_test_total(X, y)

In [ ]:
def rnn_model(features, labels, mode, params):


  num_hidden_layers = len(params['hidden_layers'])

  outputs = features["x"]

  model = params['model']

  activation = params.get("activation", tf.nn.tanh)

  print("Using activation - ", activation)

  for i in range(num_hidden_layers):


      cell = model(name = 'cell'+str(i), num_units = params['hidden_layers'][i], initializer = params['initializer'])
      outputs, state = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell, cell_bw = cell,
                                       inputs=outputs,
                                       dtype=tf.float64)

      #outputs = tf.layers.batch_normalization(outputs)
      outputs = outputs[1]

      #outputs = tf.layers.batch_normalization(outputs)

      outputs = activation(outputs)
      outputs = tf.nn.dropout(outputs, params['dropout'])

  #FLatten the output of LSTM layers
  outputs = tf.contrib.layers.flatten(outputs)

  # FC Layer
  logits = tf.layers.dense(inputs=outputs, units=params['num_classes'])

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



In [ ]:
def calc(eeg_classifier, X_train, y_train, X_test, y_test):

    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        batch_size=50,
        num_epochs=20,
        shuffle=True)

    eeg_classifier.train(
        input_fn=train_input_fn,
        steps=100)

    eval_test_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_test},
        y=y_test,
        num_epochs=1,
        shuffle=False)

    eval_train_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        num_epochs=1,
        shuffle=False)

    train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)
    test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
    print(train_results)
    print(test_results)

In [ ]:
initializers = [tf.keras.initializers.Ones(), tf.keras.initializers.Orthogonal(),
                tf.keras.initializers.RandomNormal(), tf.keras.initializers.RandomUniform(),
                tf.keras.initializers.TruncatedNormal(),
                tf.keras.initializers.Zeros(), tf.keras.initializers.glorot_normal(), tf.keras.initializers.glorot_uniform(),
                tf.keras.initializers.he_normal(), tf.keras.initializers.he_uniform(), tf.keras.initializers.lecun_normal(),
                tf.keras.initializers.lecun_uniform()]

names = ["Ones", "Orthogonal", "RandomNormal", "RandomUniform", "TruncatedNormal",
        "Zeros", "Glorot Normal", "Glorot Uniform", "He Normal", "He Uniform", "Lecun Normal", "Lecun Uniform"]

for name, init in zip(names, initializers):

    try:

        print("="*75)
        print("="*75)
        print('Trying out initializer - ', str(name))

        eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                                params = {'hidden_layers' : [64, 64], 'num_classes' : 4,
                                                          'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                         'dropout' : 0.5, 'activation' : tf.tanh,
                                                         'initializer' : init})


        calc(eeg_classifier, X_train, y_train, X_test, y_test)

    except Exception as ex:
        print("*"*75)
        print("\nException occurred - Nan loss")

Trying out initializer -  Ones
***************************************************************************

Exception occurred - Nan loss
Trying out initializer -  Orthogonal
***************************************************************************

Exception occurred - Nan loss
Trying out initializer -  RandomNormal
***************************************************************************

Exception occurred - Nan loss
Trying out initializer -  RandomUniform
***************************************************************************

Exception occurred - Nan loss
Trying out initializer -  TruncatedNormal
***************************************************************************

Exception occurred - Nan loss
Trying out initializer -  Zeros
***************************************************************************

Exception occurred - Nan loss
Trying out initializer -  Glorot Normal
***************************************************************************

Exception occurred -

###Single Script for Initializer

In [ ]:
# %%
import tensorflow as tf
import numpy as np
import importlib
import matplotlib.pyplot as plt
import preprocessing
importlib.reload(preprocessing)
from preprocessing import *
import warnings
warnings.filterwarnings('ignore')

# %%
X, y = import_data()
X_train,X_test,y_train,y_test = train_test_total(X, y)

# %%
def rnn_model(features, labels, mode, params):


  num_hidden_layers = len(params['hidden_layers'])

  outputs = features["x"]

  model = params['model']

  activation = params.get("activation", tf.nn.tanh)

  print("Using activation - ", activation)

  for i in range(num_hidden_layers):


      cell = model(name = 'cell'+str(i), num_units = params['hidden_layers'][i], initializer = params['initializer'])
      outputs, state = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell, cell_bw = cell,
                                       inputs=outputs,
                                       dtype=tf.float64)

      #outputs = tf.layers.batch_normalization(outputs)
      outputs = outputs[1]

      #outputs = tf.layers.batch_normalization(outputs)

      outputs = activation(outputs)
      outputs = tf.nn.dropout(outputs, params['dropout'])

  #FLatten the output of LSTM layers
  outputs = tf.contrib.layers.flatten(outputs)

  # FC Layer
  logits = tf.layers.dense(inputs=outputs, units=params['num_classes'])

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



# %%
def calc(eeg_classifier, X_train, y_train, X_test, y_test):

    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        batch_size=50,
        num_epochs=20,
        shuffle=True)

    eeg_classifier.train(
        input_fn=train_input_fn,
        steps=100)

    eval_test_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_test},
        y=y_test,
        num_epochs=1,
        shuffle=False)

    eval_train_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        num_epochs=1,
        shuffle=False)

    train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)
    test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
    print(train_results)
    print(test_results)

# %%
initializers = [tf.keras.initializers.Ones(), tf.keras.initializers.Orthogonal(),
                tf.keras.initializers.RandomNormal(), tf.keras.initializers.RandomUniform(),
                tf.keras.initializers.TruncatedNormal(),
                tf.keras.initializers.Zeros(), tf.keras.initializers.glorot_normal(), tf.keras.initializers.glorot_uniform(),
                tf.keras.initializers.he_normal(), tf.keras.initializers.he_uniform(), tf.keras.initializers.lecun_normal(),
                tf.keras.initializers.lecun_uniform()]

names = ["Ones", "Orthogonal", "RandomNormal", "RandomUniform", "TruncatedNormal",
        "Zeros", "Glorot Normal", "Glorot Uniform", "He Normal", "He Uniform", "Lecun Normal", "Lecun Uniform"]

for name, init in zip(names, initializers):

    try:

        print("="*75)
        print("="*75)
        print('Trying out initializer - ', str(name))

        eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                                params = {'hidden_layers' : [64, 64], 'num_classes' : 4,
                                                          'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                         'dropout' : 0.5, 'activation' : tf.tanh,
                                                         'initializer' : init})


        calc(eeg_classifier, X_train, y_train, X_test, y_test)

    except Exception as ex:
        print("*"*75)
        print("\nException occurred - Nan loss")




Trying out initializer -  Ones
***************************************************************************

Exception occurred - Nan loss
Trying out initializer -  Orthogonal
***************************************************************************

Exception occurred - Nan loss
Trying out initializer -  RandomNormal
***************************************************************************

Exception occurred - Nan loss
Trying out initializer -  RandomUniform
***************************************************************************

Exception occurred - Nan loss
Trying out initializer -  TruncatedNormal
***************************************************************************

Exception occurred - Nan loss
Trying out initializer -  Zeros
***************************************************************************

Exception occurred - Nan loss
Trying out initializer -  Glorot Normal
***************************************************************************

Exception occurred -


#Activation_Testing


In [ ]:
import tensorflow as tf
import numpy as np
import importlib
import preprocessing
importlib.reload(preprocessing)
from preprocessing import *
import warnings
warnings.filterwarnings('ignore')

In [ ]:
X, y = import_data()
X_train,X_test,y_train,y_test = train_test_total(X, y)

In [ ]:
def rnn_model(features, labels, mode, params):


  num_hidden_layers = len(params['hidden_layers'])

  outputs = features["x"]

  model = params['model']

  activation = params.get("activation", tf.nn.tanh)

  print("Using activation - ", activation)

  for i in range(num_hidden_layers):


      cell = model(name = 'cell'+str(i), num_units = params['hidden_layers'][i])
      outputs, state = tf.nn.dynamic_rnn(cell=cell,
                                       inputs=outputs,
                                       dtype=tf.float64)

      #outputs = tf.layers.batch_normalization(outputs)

      outputs = activation(outputs)
      outputs = tf.nn.dropout(outputs, params['dropout'])

  #FLatten the output of LSTM layers
  outputs = tf.contrib.layers.flatten(outputs)

  # FC Layer
  logits = tf.layers.dense(inputs=outputs, units=params['num_classes'])

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



In [ ]:
def calc(eeg_classifier, X_train, y_train, X_test, y_test):

    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        batch_size=50,
        num_epochs=20,
        shuffle=True)

    eeg_classifier.train(
        input_fn=train_input_fn,
        steps=100)

    eval_test_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_test},
        y=y_test,
        num_epochs=1,
        shuffle=False)

    eval_train_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        num_epochs=1,
        shuffle=False)

    train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)
    test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
    print(train_results)
    print(test_results)

In [ ]:
activations = [tf.nn.relu, tf.nn.crelu, tf.nn.elu, tf.nn.selu, tf.nn.softplus, tf.nn.softsign,
               tf.sigmoid, tf.tanh]

for act in activations:

    print("="*75)
    print('Trying out activation - ', str(act))

    eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                            params = {'hidden_layers' : [64, 64], 'num_classes' : 4,
                                                      'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                     'dropout' : 0.5, 'activation' : act})


    calc(eeg_classifier, X_train, y_train, X_test, y_test)

Trying out activation -  <function relu at 0x7f74c43ed480>


AttributeError: ignored

##Activation updated from chatgpt

In [ ]:
import tensorflow as tf
import numpy as np
import importlib
import preprocessing
importlib.reload(preprocessing)
from preprocessing import *
import warnings
warnings.filterwarnings('ignore')

X, y = import_data()
X_train, X_test, y_train, y_test = train_test_total(X, y)

# Convert the data to float32
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

def rnn_model(features, labels, mode, params):
    num_hidden_layers = len(params['hidden_layers'])
    outputs = features['x']
    model = params['model']
    activation = params.get('activation', tf.nn.tanh)

    print("Using activation - ", activation)

    cells = [model(units=params['hidden_layers'][i]) for i in range(num_hidden_layers)]
    rnn_layer = tf.keras.layers.RNN(cells, return_sequences=True)(outputs)
    outputs = activation(rnn_layer)
    outputs = tf.keras.layers.Dropout(rate=params['dropout'])(outputs)

    outputs = tf.keras.layers.Flatten()(outputs)
    logits = tf.keras.layers.Dense(units=params['num_classes'])(outputs)

    predictions = {
        "classes": tf.argmax(logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)(labels, logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
        with tf.GradientTape() as tape:
            logits = rnn_model(features, labels, mode, params)['logits']
            loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)(labels, logits)
        trainable_vars = rnn_model.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        optimizer.apply_gradients(zip(gradients, trainable_vars))
        train_op = optimizer.minimize(loss)
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

def calc(eeg_classifier, X_train, y_train, X_test, y_test):
    train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        batch_size=50,
        num_epochs=20,
        shuffle=True)
    eeg_classifier.train(input_fn=train_input_fn, steps=100)

    eval_test_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
        x={"x": X_test},
        y=y_test,
        num_epochs=1,
        shuffle=False)

    eval_train_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        num_epochs=1,
        shuffle=False)

    train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)
    test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
    print(train_results)
    print(test_results)

eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params={'hidden_layers': [64, 64], 'num_classes': 4,
                                                'learning_rate': 0.001, 'model': tf.keras.layers.LSTMCell,
                                                'dropout': 0.5, 'activation': tf.tanh})

calc(eeg_classifier, X_train, y_train, X_test, y_test)


Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <functio

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.10/dist-packages/keras/backend.py", line 4969, in <genexpr>
    input_ta = tuple(  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/tf_should_use.py", line 243, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.10/dist-packages/keras/backend.py", line 4969, in <genexpr>
    input_ta = tuple(  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/tf_should_use.py", line 243, in wrapped
    return _add_should_u

Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <function tanh at 0x7ff9dffcf2e0>
Using activation -  <functio

#RNN_LSTM

In [ ]:
import tensorflow as tf
import numpy as np
import importlib
import preprocessing

# Reload the preprocessing module if needed
importlib.reload(preprocessing)
from preprocessing import *
import warnings

warnings.filterwarnings('ignore')

X, y = import_data()
X_train, X_test, y_train, y_test = train_test_total(X, y)

def rnn_model(features, labels, mode, params):
    num_hidden_layers = len(params['hidden_layers'])
    input_layer = features["x"]

    cell1 = tf.keras.layers.LSTM(params['hidden_layers'][0], return_sequences=True, name='cell1')
    outputs = cell1(input_layer)
    outputs = tf.nn.relu(outputs)
    outputs = tf.keras.layers.Dropout(rate=0.5)(outputs)

    for i in range(num_hidden_layers - 1):
        return_sequences = i < num_hidden_layers - 2  # Set return_sequences=True for all layers except the last
        cell = tf.keras.layers.LSTM(params['hidden_layers'][i + 1], return_sequences=return_sequences, name='cell' + str(i + 2))
        outputs = cell(outputs)
        outputs = tf.nn.relu(outputs)

    # Flatten the output of LSTM layers
    outputs = tf.keras.layers.Flatten()(outputs)

    logits = tf.keras.layers.Dense(units=params['num_classes'])(outputs)

    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    labels = tf.reshape(labels, [-1])  # Reshape labels to a 1D tensor
    loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels, logits))
    tf.summary.scalar('loss', loss)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
        train_vars = tf.compat.v1.trainable_variables()
        grads = tf.gradients(loss, train_vars)
        grads_and_vars = zip(grads, train_vars)
        train_op = optimizer.apply_gradients(grads_and_vars)
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

eeg_classifier = tf.estimator.Estimator(
    model_fn=rnn_model,
    model_dir="/model/",
    params={'hidden_layers': [32, 32], 'num_classes': 4, 'learning_rate': 0.001}
)

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices(({"x": X_train}, y_train))
    dataset = dataset.batch(64).repeat(20).shuffle(buffer_size=1000)
    return dataset

logging_hook = tf.estimator.LoggingTensorHook(
    tensors={"probabilities": "softmax_tensor"},
    every_n_iter=50
)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook]
)

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices(({"x": X_test}, y_test))
    dataset = dataset.batch(64)
    return dataset

eval_results = eeg_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)


Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use standard file utilities to get mtimes.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


#Bidirectional

In [ ]:
# %%
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import importlib
import preprocessing
importlib.reload(preprocessing)
from preprocessing import *
import warnings
warnings.filterwarnings('ignore')

# %%
X, y = import_data()
X_train,X_test,y_train,y_test = train_test_total(X, y)

# %%
def rnn_model(features, labels, mode, params):


  num_hidden_layers = len(params['hidden_layers'])

  outputs = features["x"]

  model = params['model']

  activation = params.get("activation", tf.nn.tanh)

  print("Using activation - ", activation)

  for i in range(num_hidden_layers):


      cell = model(name = 'cell'+str(i), num_units = params['hidden_layers'][i])
      outputs, state = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell, cell_bw = cell,
                                       inputs=outputs,
                                       dtype=tf.float64)

      #outputs = tf.layers.batch_normalization(outputs)
      outputs = outputs[1]
      outputs = activation(outputs)
      outputs = tf.nn.dropout(outputs, params['dropout'])

  print(type(outputs))
  print(outputs.shape)
  #FLatten the output of LSTM layers
  outputs = tf.keras.layers.Flatten()(outputs)

  print(type(outputs))
  print(outputs.shape)
  # FC Layer
  logits = tf.layers.dense(inputs=outputs, units=params['num_classes'])

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)




# %%
def calc(eeg_classifier, X_train, y_train, X_test, y_test):

    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        batch_size=50,
        num_epochs=20,
        shuffle=True)

    eeg_classifier.train(
        input_fn=train_input_fn,
        steps=100)

    eval_test_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_test},
        y=y_test,
        num_epochs=1,
        shuffle=False)

    eval_train_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        num_epochs=1,
        shuffle=False)

    train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)
    test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
    print(train_results)
    print(test_results)

# %%
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [64, 64, 64], 'num_classes' : 4,
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

# %%
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [64, 64], 'num_classes' : 4,
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

# %%
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [128, 128], 'num_classes' : 4,
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

# %%
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [256, 256], 'num_classes' : 4,
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

# %%
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [128, 128], 'num_classes' : 4,
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.BasicRNNCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

# %%
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                        params = {'hidden_layers' : [128, 128], 'num_classes' : 4,
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.GRUCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

# %%





Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 64000)


Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


Using activation -  <function tanh at 0x7f44c9f75990>


Instructions for updating:
Use tf.keras instead.


...

{'accuracy': 0.7535885, 'loss': 0.6606881, 'global_step': 100}
{'accuracy': 0.54, 'loss': 0.9481491, 'global_step': 100}
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)


{'accuracy': 0.75239235, 'loss': 0.6225511, 'global_step': 100}
{'accuracy': 0.66, 'loss': 0.812376, 'global_step': 100}
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 256)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 256000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 256)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 256000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 256)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 256000)


{'accuracy': 0.761563, 'loss': 0.6178331, 'global_step': 100}
{'accuracy': 0.6, 'loss': 0.9036742, 'global_step': 100}
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


{'accuracy': 0.66586924, 'loss': 1.1482065, 'global_step': 100}
{'accuracy': 0.38, 'loss': 2.2814727, 'global_step': 100}
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
Using activation -  <function tanh at 0x7f44c9f75990>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 128)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 128000)
{'accuracy': 0.75, 'loss': 0.6398369, 'global_step': 100}
{'accuracy': 0.56, 'loss': 1.0955027, 'global_step': 100}
